In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import category_encoders as ce
import functools

from keras.layers import Concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D
# from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import backend as K
# from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher

# Lectura de datos

In [2]:
data = pd.read_csv('D:/Visual Studio Code/HK 6/Data Mining/RASFF-predictions-dashboard/Copia csvs antiguos/splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
df = data
df

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
59976,2014-10-27,France,milk and milk products,food,serious,recall from consumers,distribution to other member countries,microbial contaminants (other),France,Germany,Commission Services
48103,2016-02-23,Italy,cephalopods and products thereof,food,serious,import not authorised,product not (yet) placed on the market,metals,India,,Italy
33200,2017-07-13,Netherlands,cereals and bakery products,food,undecided,withdrawal from recipient(s),distribution to other member countries,food additives and flavourings,United States,Austria,
119457,2007-11-08,Slovakia,confectionery,food,undecided,withdrawal from the market,distribution on the market (possible),food additives and flavourings,Spain,Slovakia,Czech Republic
89098,2011-11-28,Germany,cereals and bakery products,food,undecided,withdrawal from the market,distribution to other member countries,mycotoxins,Sweden,Slovakia,
...,...,...,...,...,...,...,...,...,...,...,...
96666,2011-01-17,Germany,non-alcoholic beverages,food,undecided,withdrawal from the market,distribution on the market (possible),foreign bodies,Germany,Austria,
121940,2007-05-31,United Kingdom,fats and oils,food,undecided,withdrawal from the market,distribution on the market (possible),organoleptic aspects,United Kingdom,Greece,
36099,2017-04-27,Belgium,fruits and vegetables,food,serious,withdrawal from the market,distribution to other member countries,metals,France,France,INFOSAN
102067,2010-04-01,Germany,food contact materials,fcm,undecided,withdrawal from the market,distribution on the market (possible),metals,China,Slovakia,Commission Services


In [3]:
mask = (df['DATE_CASE'] >= '2004-01-01') & (df['DATE_CASE'] <= '2018-12-31')
df = df.loc[mask]
df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
df['DATE_CASE'] = df.DATE_CASE.dt.month

C:\Users\Locc\AppData\Local\Temp\ipykernel_14896\335135510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
C:\Users\Locc\AppData\Local\Temp\ipykernel_14896\335135510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] = df.DATE_CASE.dt.month


In [4]:
features = [0,1,2,6,7,8]
target = [5]
X = df.iloc[:,features]
y = df.iloc[:,target]

In [5]:
X

,DATE_CASE,NOT_COUNTRY,PROD_CAT,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN
59976,10,France,milk and milk products,distribution to other member countries,microbial contaminants (other),France
48103,2,Italy,cephalopods and products thereof,product not (yet) placed on the market,metals,India
33200,7,Netherlands,cereals and bakery products,distribution to other member countries,food additives and flavourings,United States
119457,11,Slovakia,confectionery,distribution on the market (possible),food additives and flavourings,Spain
89098,11,Germany,cereals and bakery products,distribution to other member countries,mycotoxins,Sweden
...,...,...,...,...,...,...
96666,1,Germany,non-alcoholic beverages,distribution on the market (possible),foreign bodies,Germany
121940,5,United Kingdom,fats and oils,distribution on the market (possible),organoleptic aspects,United Kingdom
36099,4,Belgium,fruits and vegetables,distribution to other member countries,metals,France
102067,4,Germany,food contact materials,distribution on the market (possible),metals,China


In [6]:
y

,ACTION_TAKEN
59976,recall from consumers
48103,import not authorised
33200,withdrawal from recipient(s)
119457,withdrawal from the market
89098,withdrawal from the market
...,...
96666,withdrawal from the market
121940,withdrawal from the market
36099,withdrawal from the market
102067,withdrawal from the market


# Selección de codificación

In [7]:
ency = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
ency.fit(y.values)
y_one_hot = ency.transform(y.values)

In [8]:
'''encx = OrdinalEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

'encx = OrdinalEncoder()\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)'

In [9]:
'''encx = OneHotEncoder(handle_unknown='ignore', sparse = False)
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

"encx = OneHotEncoder(handle_unknown='ignore', sparse = False)\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)"

In [10]:
encx  = ce.BaseNEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)

c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWa

In [11]:
X_train, X_val, y_train, y_val = train_test_split(x_one_hot, y_one_hot, test_size=0.2)

# Pruebas con mlp básico

In [12]:
accs = []
for i in range (1,6):
    K.clear_session()
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=len(X_train.values[1])))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation = "softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam",  metrics=['categorical_accuracy'])
    hist = model.fit(X_train,y_train, epochs= 15,validation_data=(X_val, y_val), batch_size = 64)
    accs.append(hist.history['val_categorical_accuracy'][9])
    model.save ("model"+str(i)+".h5")

c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 51s 29ms/step - categorical_accuracy: 0.4272 - loss: 1.8940 - val_categorical_accuracy: 0.5952 - val_loss: 1.3160
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 47s 29ms/step - categorical_accuracy: 0.5959 - loss: 1.3044 - val_categorical_accuracy: 0.6639 - val_loss: 1.1052
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 50s 30ms/step - categorical_accuracy: 0.6585 - loss: 1.0905 - val_categorical_accuracy: 0.6922 - val_loss: 0.9922
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 55s 33ms/step - categorical_accuracy: 0.6930 - loss: 0.9656 - val_categorical_accuracy: 0.7176 - val_loss: 0.9244
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 54s 33ms/step - categorical_accuracy: 0.7209 - loss: 0.8745 - val_categorical_accuracy: 0.7353 - val_loss: 0.8675
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 59s 35ms/step - categorical_accuracy: 0.7402 - loss: 0.7976 - val_categorical_accuracy: 0.7449 - val_loss: 0.8355
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 55s 33ms/step - catego

Epoch 1/15


c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1647/1647 ━━━━━━━━━━━━━━━━━━━━ 58s 34ms/step - categorical_accuracy: 0.4215 - loss: 1.9137 - val_categorical_accuracy: 0.5932 - val_loss: 1.3352
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 58s 35ms/step - categorical_accuracy: 0.5934 - loss: 1.3175 - val_categorical_accuracy: 0.6515 - val_loss: 1.1215
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 46s 28ms/step - categorical_accuracy: 0.6527 - loss: 1.1083 - val_categorical_accuracy: 0.6947 - val_loss: 1.0011
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accuracy: 0.6907 - loss: 0.9776 - val_categorical_accuracy: 0.7170 - val_loss: 0.9196
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accuracy: 0.7190 - loss: 0.8788 - val_categorical_accuracy: 0.7311 - val_loss: 0.8701
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 28ms/step - categorical_accuracy: 0.7390 - loss: 0.8092 - val_categorical_accuracy: 0.7466 - val_loss: 0.8407
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accur

...

c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1647/1647 ━━━━━━━━━━━━━━━━━━━━ 46s 27ms/step - categorical_accuracy: 0.4243 - loss: 1.9093 - val_categorical_accuracy: 0.5982 - val_loss: 1.3128
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 44s 27ms/step - categorical_accuracy: 0.5964 - loss: 1.3106 - val_categorical_accuracy: 0.6551 - val_loss: 1.1276
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 28ms/step - categorical_accuracy: 0.6556 - loss: 1.1007 - val_categorical_accuracy: 0.6959 - val_loss: 0.9955
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accuracy: 0.6939 - loss: 0.9715 - val_categorical_accuracy: 0.7204 - val_loss: 0.9198
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accuracy: 0.7200 - loss: 0.8733 - val_categorical_accuracy: 0.7301 - val_loss: 0.8816
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 44s 27ms/step - categorical_accuracy: 0.7408 - loss: 0.8031 - val_categorical_accuracy: 0.7485 - val_loss: 0.8344
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accur

Epoch 1/15


c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1647/1647 ━━━━━━━━━━━━━━━━━━━━ 46s 27ms/step - categorical_accuracy: 0.4243 - loss: 1.9025 - val_categorical_accuracy: 0.6012 - val_loss: 1.3118
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accuracy: 0.5955 - loss: 1.3137 - val_categorical_accuracy: 0.6570 - val_loss: 1.1130
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 44s 27ms/step - categorical_accuracy: 0.6558 - loss: 1.0987 - val_categorical_accuracy: 0.6959 - val_loss: 1.0027
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 46s 28ms/step - categorical_accuracy: 0.6966 - loss: 0.9626 - val_categorical_accuracy: 0.7197 - val_loss: 0.9168
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 53s 32ms/step - categorical_accuracy: 0.7193 - loss: 0.8738 - val_categorical_accuracy: 0.7336 - val_loss: 0.8706
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 28ms/step - categorical_accuracy: 0.7394 - loss: 0.7997 - val_categorical_accuracy: 0.7499 - val_loss: 0.8319
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 45s 27ms/step - categorical_accur

In [13]:
suma = 0
for i in accs:
    suma = suma + i
print(suma/5)

0.7710011720657348
